# Pra-Pemrosesan Data Teks

Salah satu Prinsip Utama untuk dipahami selama Pra-pemrosesan Data adalah memiliki Ide yang jelas tentang bagaimana tampilan data Input dan dan bagaimana kita ingin output akhir terlihat.

Langkah-langkah yang diikuti untuk pre-processing adalah sebagai berikut :

- Memahami Format Data
- Menyimpan data Topan ke dalam <i>Dictionary</i>
- Mengubah <i>Dictionary</i> menjadi <i>Dataframe</i>
- Restrukturisasi Kolom dan membuatnya mudah dibaca
- Mengganti Nilai Sentinel dan Menghapus String Kosong
- Menghapus Spasi yang Tidak Diinginkan dan Mengindeks ulang <i>dataframe</i>
- Simpan Dataframe ini ke File CSV


### Memahami Format Data

Mari kita lihat Modifikasi Format CSV yang digunakan oleh Tim HURDAT2 :

In [ ]:
from IPython.display import IFrame
IFrame("http://www.nhc.noaa.gov/data/hurdat/hurdat2-format-atlantic.pdf", width=950, height=600)

File telah diunduh dan sekarang mari kita baca file tersebut.

In [ ]:
# Membuka File Data
atlantic = open("data/hurdat2-1851-2018-120319.txt", "r")
atlantic_raw = atlantic.read()

# Menjalankan penghitung (counter) untuk memeriksa dua huruf pertama dari Dokumen
import io
from collections import Counter

c = Counter()
for line in io.StringIO(atlantic_raw):
    c[line[:2]] += 1
#Printing Counter Output
print(c)



Memahami Apa Arti Output penghitung (<i>counter</i>) :

* AL : Jumlah Badai Atlantik dari 1851-2018
* 18 : Jumlah Entri pada Abad ke-19 ( 1851 - 1899)
* 19 : Jumlah Entri pada Abad ke-20 ( 1900 - 1999)
* 20 : Jumlah Entri Abad 21 ( 2000 - 2018)


### Menyimpan data Topan dalam <i>dictionary</i>

Sekarang mari kita buat <i>Dictionary</i> untuk menyimpan data Topan sesuai dengan namanya.

In [ ]:
import io

# Buat dictionary untuk Menyimpan Semua Data Topan Sesuai dengan namanya
atlantic_storms_r = []
atlantic_storm_r = {'header': None, 'data': []}

for i, line in enumerate(io.StringIO(atlantic_raw)):
    if line[:2] == 'AL':
        atlantic_storms_r.append(atlantic_storm_r.copy())
        atlantic_storm_r['header'] = line
        atlantic_storm_r['data'] = []
    else:
        atlantic_storm_r['data'].append(line)
# Menghapus Elemen Pertama dari List dan Menyimpan Lainnya.
atlantic_storms_r = atlantic_storms_r[1:]
# Jumlah Topan Atlantik
len(atlantic_storms_r)

### Mengubah <i>dictionary</i> menjadi <i>Dataframe</i>

In [ ]:
# Mengkonversi Dictionary menjadi Pandas Dataframe

import pandas as pd

atlantic_storm_dfs = []
for storm_dict in atlantic_storms_r:
    storm_id, storm_name, storm_entries_n = storm_dict['header'].split(",")[:3]
    # menghapus baris baru atau newline ('\n'), dan memisahkan data dengan koma(,)
    data = [[entry.strip() for entry in datum[:-1].split(",")] for datum in storm_dict['data']]
    frame = pd.DataFrame(data)
    frame['id'] = storm_id
    frame['name'] = storm_name
    atlantic_storm_dfs.append(frame)
    
# Menampilkan dataframe Topan 
atlantic_storm_dfs[0]

In [ ]:
# Menggabungkan Semua Data Topan menjadi satu
atlantic_storms = pd.concat(atlantic_storm_dfs)
len(atlantic_storms)

### Restrukturisasi Kolom dan membuatnya mudah dibaca

In [ ]:
# Restrukturisasi kolom pada Dataframe
atlantic_storms = atlantic_storms.reindex(columns=atlantic_storms.columns[-2:] | atlantic_storms.columns[:-2]) 
# Mencetak 5 baris pertama
atlantic_storms.head()

In [ ]:
#Menampilkan kolom-kolom yang ada di dataset
atlantic_storms.columns

In [ ]:
# Memproses kolom agar lebih mudah dibaca
atlantic_storms.columns = [
        "id",
        "name",
        "date",
        "hours_minutes",
        "record_identifier",
        "status_of_system",
        "latitude",
        "longitude",
        "maximum_sustained_wind_knots",
        "maximum_pressure",
        "34_kt_ne",
        "34_kt_se",
        "34_kt_sw",
        "34_kt_nw",
        "50_kt_ne",
        "50_kt_se",
        "50_kt_sw",
        "50_kt_nw",
        "64_kt_ne",
        "64_kt_se",
        "64_kt_sw",
        "64_kt_nw",
        "na"
]
del atlantic_storms['na']
pd.set_option("max_columns", None)

In [ ]:
# Melihat dataframe : 
atlantic_storms.head()

### Mengganti Nilai Sentinel dan Membuang String kosong

Sekarang kita telah menyelesaikan sebagian besar Parsing , Mari kita lakukan beberapa perbaikan terakhir dengan mengubah nilai sentinel yaitu '-999' menjadi NaN ( Bukan angka )

In [ ]:
# Mengganti semua nilai Sentinel (-999 ) dengan nan
atlantic_storms.iloc[0]['34_kt_sw']

# Kita menggunakan Numpy ( Numerical Python ) untuk mengganti nilai Sentinel.
import numpy as np
atlantic_storms = atlantic_storms.replace(to_replace='-999', value=np.nan)
atlantic_storms.iloc[0]['34_kt_sw']

In [ ]:
# Mengecek tipe data dari setiap Kolom
atlantic_storms.dtypes

In [ ]:
atlantic_storms['record_identifier'].value_counts()

Mengganti semua Empty Strings dengan NaN

In [ ]:
# Mengganti Semua String Kosong dengan nilai NaN
atlantic_storms = atlantic_storms.replace(to_replace="", value=np.nan)
atlantic_storms['record_identifier'].value_counts(dropna=False)

In [ ]:
#Melihat dataframe : 
atlantic_storms.head()

### Menghapus Spasi yang Tidak Diinginkan dan Mengindeks Ulang <i>dataframe</i>

In [ ]:
# Perbaikan terakhir

# Lepaskan Spasi yang Tidak Diinginkan dari nama
atlantic_storms['name'] = atlantic_storms['name'].map(lambda n: n.strip()) 

#ReIndex
atlantic_storms.index = range(len(atlantic_storms.index))
atlantic_storms.index.name = "index"

### Menyimpan Dataframe ini ke File CSV

Mari kita simpan Dataframe ke dalam file CSV yang akan kita gunakan untuk Menganotasi Data.

In [ ]:
atlantic_storms.tail()

In [ ]:
atlantic_storms.to_csv("atlantic.csv")

## License

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).